# AI Tools

## Project - Airlines AI Assistant

We will make an AI Customer Support assistant for an Airline with pre-fixed prices

In [1]:
# Importing the Libraries

import requests
import json
import time
import re
import ollama
from typing import List
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
import gradio as gr
import uuid

In [2]:
# Initializing System Messages and ollama details

system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME="qwen3"

### Tools
Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

In [3]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

DB_HOST="localhost"
DB_PORT="3306"
DB_NAME="personal_DB"
DB_USER="Alex"
DB_PASSWORD="Alex@$14798|<</>>"

connection_string = (
    f"mysql+pymysql://{DB_USER}:{quote_plus(DB_PASSWORD)}@"
    f"{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(connection_string, echo=True)

In [4]:
def get_ticket_price_db(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with engine.connect() as conn:
        result = conn.execute(text(f'SELECT price FROM personal_DB.prices WHERE city = "{city.lower()}"'))
        result = result.fetchall()
        conn.close()
        return f"Ticket price to {city} is ₹ {result[0][0]}" if result else "No price data available for this city"

In [5]:
get_ticket_price_db("Chennai")

DATABASE TOOL CALLED: Getting price for Chennai
2025-12-26 15:32:35,619 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-12-26 15:32:35,620 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-26 15:32:35,621 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-12-26 15:32:35,622 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-26 15:32:35,623 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-12-26 15:32:35,623 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-26 15:32:35,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-26 15:32:35,627 INFO sqlalchemy.engine.Engine SELECT price FROM personal_DB.prices WHERE city = "chennai"
2025-12-26 15:32:35,627 INFO sqlalchemy.engine.Engine [generated in 0.00155s] {}
2025-12-26 15:32:35,637 INFO sqlalchemy.engine.Engine ROLLBACK


'Ticket price to Chennai is ₹ 399.0'

In [6]:
# There's a particular dictionary structure that's required to describe our function:

# Tool Name
price_function = {
        "name": "get_ticket_price",  # Name of the function
        # This is important since its passed to LLM so that when it should call the tool we have to mention it clearly, its like system prompt for tools. Be clear as possible 
        "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
        "parameters": {
            # Here we have to give the details of the parameters, here we have one parameter so mentioning that here
            "type": "object",
            "properties":{
                "destination_city":{
                    # Mentioning the data type and where and how this input is used
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            # we have to mention what are the must required parameters of the function(tool)
            "required":["destination_city"],
            # It restricts the input object to allow only the explicitly defined properties — no extras.
            # like only "destination_city": "Berlin" this is allowed they can't send extra prameters like "class": "economy"
            "additionalProperties": False,
        }
    }

In [7]:
# And this is included in a list of tools:

tools = [
    {
        "type": "function",  # Since we are calling the function we have to mention it
        "function": price_function,
    }
]

In [8]:
# We have to write that function handle_tool_call:

def multi_handle_tool_calls_db(message):
    responses = [] # using list to append all tools responses
    cities = [] # using list to append all cities fetched from tool calls
    for tool_call in message["tool_calls"]:
        if tool_call["function"]["name"] == "get_ticket_price":
            arguments = tool_call["function"]["arguments"]
            if isinstance(arguments, str):
                arguments = json.loads(arguments)
            city = arguments.get('destination_city')
            price = get_ticket_price_db(city)
        
            tool_call_id = tool_call.get("id", str(uuid.uuid4()))
            response = {
                "role": "tool",
                "content": json.dumps({"destination_city": city, "price": price}),
                "tool_call_id": tool_call_id
            }
            responses.append(response)
            cities.append(city)
    return responses, cities

In [9]:
# For tools we are going to use ollama package instead of APIs(Since APIs don't support tools)
# This tools support will be provides by only specific models so we switch to Qwen from Mistral

def multi_chat_db(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    result = ""
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=messages,
            tools=tools,
            stream=False,
            options={
                "temperature": 0.8
            }
        )
        print(response, "\n")

        msg = response.get("message", {})
        while msg.get("tool_calls"): # this will allow the LLM to run all tool calls it makes
            print(msg.get("tool_calls"))
            tool_response, city = multi_handle_tool_calls_db(msg)
            messages.append(msg)
            messages.extend(tool_response) # we are recieving a list of response so extend it instead of appending it
            response = ollama.chat(
                model=MODEL_NAME,
                messages=messages,
                stream=False,
                options={
                    "temperature": 0.8
                }
            )

            msg = response.get("message", {})
            
        if msg.get("content"):
            return {"role": "assistant", "content": msg["content"]}
        
        return {"role": "assistant", "content": "I'm not sure how to respond."}
                    
    except Exception as e:
        print("Exception: ", repr(e))

In [10]:
# Chat Interface

gr.ChatInterface(fn=multi_chat_db, chatbot=gr.Chatbot(type="messages")).launch(inline=False)

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# Let's build a multi-model App
We can use a Free image generation API called pollinations.ai and SubNP, the image generation model we can choose, to make us some images

Let's put this in a function called artist.


In [11]:
import base64
from io import BytesIO
from PIL import Image

In [12]:
# from PIL import Image
# from IPython.display import display
# from io import BytesIO

# # Image parameters
# prompt = 'Generate image of Joseph Stalin waving the red flag in front of the red army soviet'
# width = 1024
# height = 1024inline=False
# seed = 42   # Each seed generates a new image variation
# model = 'flux' # Using 'flux' as default if model is not provided

# # Construct the image URL
# image_url = f"https://pollinations.ai/p/{prompt}?width={width}&height={height}&seed={seed}&model={model}"

# # Fetch and display the image without saving
# response = requests.get(image_url)
# img = Image.open(BytesIO(response.content))
# display(img)

In [13]:
def artist(city):
    model = 'flux' # Using 'flux' as default if model is not provided
    prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style"
    width = 1024
    height = 1024
    seed = 42   # Each seed generates a new image variation

    # Construct the image URL
    image_url = f"https://pollinations.ai/p/{prompt}?width={width}&height={height}&seed={seed}&model={model}"

    # Fetch and display the image without saving
    response = requests.get(image_url)
    
    return Image.open(BytesIO(response.content))

In [14]:
image = artist("India")

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f8d22d70950>

In [16]:
display(image)

NameError: name 'image' is not defined

We are using Audio models also for this multi Agent, so for this we use gTTS 

gTTS (Google Text-to-Speech), a Python library and CLI tool to interface with Google Translate's text-to-speech API.

For different Languages refer this -->https://gtts.readthedocs.io/en/latest/module.html#languages-gtts-lang

Optional, There is a free API for this called TTSMP3 api, if required we can use that for different modulation

In [17]:
# Install it if its not installed

# !pip install gTTS

In [18]:
from IPython.display import Audio
from gtts import gTTS
import io

def talker(message):
    # Create the TTS object
    tts = gTTS(text=message, lang='en')
    
    # Save to a file
    # tts.save("output.mp3")
    
    mp3_fp = io.BytesIO()
    tts.write_to_fp(mp3_fp)
    
    # Seek to the start so IPython can read it
    mp3_fp.seek(0)
    
    # Play the audio directly in JupyterLab
    return Audio(mp3_fp.read())

In [19]:
talker("mangekyou sharingan")

In [20]:
def talker(message):
    # Create the TTS object
    tts = gTTS(text=message, lang='en')
    
    # Save to a file
    # tts.save("output.mp3")
    
    mp3_fp = io.BytesIO()
    tts.write_to_fp(mp3_fp)
    
    # Seek to the start so IPython can read it
    mp3_fp.seek(0)
    return mp3_fp.read()

### Our Agent Framework
The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

* Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
* The ability for LLMs to use Tools to give them additional capabilities
* The 'Agent Environment' which allows Agents to collaborate
* An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
* The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory

In [21]:
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history

    result = ""
    image = None
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=messages,
            tools=tools,
            stream=False,
            options={
                "temperature": 0.8
            }
        )
        print(response, "\n")

        msg = response.get("message", {})
        while msg.get("tool_calls"): # this will allow the LLM to run all tool calls it makes
            print(msg.get("tool_calls"))
            tool_response, cities = multi_handle_tool_calls_db(msg)
            messages.append(msg)
            messages.extend(tool_response) # we are recieving a list of response so extend it instead of appending it
            response = ollama.chat(
                model=MODEL_NAME,
                messages=messages,
                stream=False,
                options={
                    "temperature": 0.8
                }
            )

            if cities:
                image = artist(cities[0])
            msg = response.get("message", {})
            
        if msg.get("content"):
            cleaned_reply = re.sub(r"<think>.*?</think>", "", msg["content"], flags=re.DOTALL).strip()
            audio = talker(cleaned_reply)
            display(audio)
            history.append({"role": "assistant", "content": msg["content"]})
            return history, audio, image
        
        history.append({"role": "assistant", "content": "I'm not sure how to respond."})
        return history, image
                    
    except Exception as e:
        print("Exception: ", repr(e))
        history.append({"role": "assistant", "content": f"An error occurred: {str(e)}"})
        return history, None

### The 3 types of Gradio UI

`gr.Interface` is for standard, simple UIs

`gr.ChatInterface` is for standard ChatBot UIs

`gr.Blocks` is for custom UIs where you control the components and the callbacks

In [22]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

# with gr.Blocks() as ui:
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500, type="messages")
#         image_output = gr.Image(height=500)
#     with gr.Row():
#         entry = gr.Textbox(label="Chat with our AI Assistant:")
#     with gr.Row():
#         clear = gr.Button("Clear")

#     def do_entry(message, history):
#         history += [{"role":"user", "content":message}]
#         return "", history

#     entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
#         chat, inputs=chatbot, outputs=[chatbot, image_output]
#     )
#     clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# ui.launch(inbrowser=True, inline=False)


In [23]:
# Callbacks (along with the chat() function above)

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True, auth=("root", "root"))

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
